# Conecting & Configuring the SQLite

In [1]:
%load_ext sql


In [2]:
%sql sqlite:///database_sales.db


# Exploratory and Test


In [3]:
%%sql
-- Raw Data
SELECT *
FROM sales
LIMIT 10;

Running query in 'sqlite:///database_sales.db'

order_number,date,warehouse,client_type,product_line,quantity,unit_price,total,payment,payment_fee
N1,2021-06-01T00:00:00.000Z,North,Retail,Breaking system,9,19.29,173.61,Cash,0.0
N2,2021-06-01T00:00:00.000Z,North,Retail,Suspension & traction,8,32.93,263.45,Credit card,0.03
N3,2021-06-01T00:00:00.000Z,North,Wholesale,Frame & body,16,37.84,605.44,Transfer,0.01
N4,2021-06-01T00:00:00.000Z,North,Wholesale,Suspension & traction,40,37.37,1494.8,Transfer,0.01
N5,2021-06-01T00:00:00.000Z,North,Retail,Frame & body,6,45.44,272.61,Credit card,0.03
N6,2021-06-02T00:00:00.000Z,North,Retail,Frame & body,1,40.41,40.41,Credit card,0.03
N7,2021-06-02T00:00:00.000Z,North,Retail,Miscellaneous,6,20.28,121.66,Credit card,0.03
N8,2021-06-03T00:00:00.000Z,North,Retail,Electrical system,9,20.5,184.54,Credit card,0.03
N9,2021-06-03T00:00:00.000Z,North,Retail,Suspension & traction,5,36.18,180.91,Credit card,0.03
N10,2021-06-03T00:00:00.000Z,North,Retail,Electrical system,5,28.33,141.67,Credit card,0.03


In [4]:
%%sql
-- Testing to see about my date date pun intended 

SELECT DISTINCT STRFTIME('%m',date) AS Month
FROM sales;

Running query in 'sqlite:///database_sales.db'

Month
06
07
08


In [5]:
%%sql
-- Test net revenue

SELECT product_line, date, warehouse, payment_fee,
       SUM(total) * (1-payment_fee) AS net_revenue
FROM sales
GROUP BY product_line,date, warehouse,payment_fee
LIMIT 5;

Running query in 'sqlite:///database_sales.db'

product_line,date,warehouse,payment_fee,net_revenue
Breaking system,2021-06-01T00:00:00.000Z,Central,0.03,217.6486
Breaking system,2021-06-01T00:00:00.000Z,North,0.0,173.61
Breaking system,2021-06-02T00:00:00.000Z,Central,0.03,131.8424
Breaking system,2021-06-02T00:00:00.000Z,West,0.03,48.44179999999999
Breaking system,2021-06-04T00:00:00.000Z,Central,0.03,76.7658


# Result

In [6]:
%%sql 

WITH month_cte AS (
	SELECT order_number,
		STRFTIME('%m',date) as month_raw
	FROM sales
	
)

SELECT product_line, 
	CASE WHEN m.month_raw = '06' THEN 'June'
	WHEN m.month_raw = '07' THEN 'July'
	WHEN m.month_raw = '08' THEN 'August'
	ELSE 'ERROR' END AS month,
	warehouse,
    ROUND(CAST(SUM(total * (1 - payment_fee)) AS REAL),2) AS net_revenue
FROM sales
INNER JOIN month_cte AS m
ON sales.order_number = m.order_number
WHERE client_type = 'Wholesale' 
GROUP BY product_line,month,warehouse
ORDER BY product_line,month,net_revenue DESC
LIMIT 10;

Running query in 'sqlite:///database_sales.db'

product_line,month,warehouse,net_revenue
Breaking system,August,Central,3009.1
Breaking system,August,West,2475.71
Breaking system,August,North,1753.19
Breaking system,July,Central,3740.94
Breaking system,July,West,3030.39
Breaking system,July,North,2568.55
Breaking system,June,Central,3648.14
Breaking system,June,North,1472.93
Breaking system,June,West,1200.64
Electrical system,August,North,4673.99



# Apendix
## Context 

You're working for a company that sells motorcycle parts, and they've asked with some help in analyzing their sales data!

They operate three warehouses in the area, selling both retail and wholesale. They offer a variety of parts and accept credit card, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors want to gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses. You have been tasked with calculating net revenue for each product line, grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

They have provided you with access to their database, which contains the following table called `sales`:

| Column | Data type | Description |
|--------|-----------|-------------|
| `order_number` | `TEXT` | Unique order number. |
| `date` | `TEXT` | Date of the order, from June to August 2021. |
| `warehouse` | `TEXT` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type` | `TEXT` | Whether the order was `Retail` or `Wholesale`. |
| `product_line` | `TEXT` | Type of product ordered. |
| `quantity` | `INTEGER` | Number of products ordered. | 
| `unit_price` | `REAL` | Price per product (dollars). |
| `total` | `REAL` | Total price of the order (dollars). |
| `payment` | `REAL` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`. |
| `payment_fee` | `REAL` | Percentage of `total` charged as a result of the `payment` method. |


Your query output should be presented in the following format:

| `product_line` | `month` | `warehouse` |	`net_revenue` |
|----------------|-----------|----------------------------|--------------|
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_two | --- | --- | --- |
| ... | ... | ... | ... |

